# Candles
## Get OHLCV for multiple time frames

The basic syntax is as follows


In [7]:
from backintime import FuturesStrategy
from backintime.timeframes import Timeframes as tf

class DummyStrategy(FuturesStrategy):
    candle_timeframes = {tf.M1, tf.M5} # Declare that 1min and 5min are required
    def tick(self):
        m1 = self.candles.get(tf.M1)  # get by 1min timeframe
        m5 = self.candles.get(tf.M5)  # get by 5min timeframe

In order to actually get the results we need to run backtesting. Here's the full code with imports, feed and other necessary setup. OHLCV data is stored in pandas DataFrames (just so it's easy to see its content). Note that although the open/close times for 5min candles are equal for each 5 subsequent rows, its OHLCV values are different since 5mins data is upsampled from 1mins. Thus, its OHLCV is updated each minute and the last row (of each 5) holds its final state. In order for this feature to work, the feed's timeframe MUST be the same or smaller (it's 1min in this case).

In [9]:
import os
from datetime import datetime
from backintime import FuturesStrategy, run_backtest
from backintime.trading_strategy import AbstractStrategyFactory
from backintime.timeframes import Timeframes as tf
from backintime.data.csv import CSVCandlesFactory, CSVCandlesSchema
from backintime.utils import PREFETCH_SINCE

import pandas as pd  # going to save the results in DataFrame
pd.set_option('display.expand_frame_repr', False)
# NOTE: the first column, 'upd_time', does not belong to the original OHLCV values and included here for clarity
m1 = pd.DataFrame(columns=['upd_time', 'open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time'])
m5 = pd.DataFrame(columns=['upd_time', 'open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time'])

class DummyStrategy(FuturesStrategy):
    candle_timeframes = {tf.M1, tf.M5}  # 1min and 5mins
    def tick(self):
        # Retrieve OHLCV values for 1min and 5mins
        ohlcv_m1 = self.candles.get(tf.M1)
        ohlcv_m5 = self.candles.get(tf.M5)
        # This is just to view the results
        m1.loc[len(m1)] = [
            ohlcv_m1.close_time,   # Add time of update: close time of the current 1min
            ohlcv_m1.open_time,
            ohlcv_m1.open,
            ohlcv_m1.high,
            ohlcv_m1.low,
            ohlcv_m1.close,
            ohlcv_m1.volume,
            ohlcv_m1.close_time
        ]
        m5.loc[len(m5)] = [
            ohlcv_m1.close_time,   # Add time of update: close time of the current 1min
            ohlcv_m5.open_time,
            ohlcv_m5.open,
            ohlcv_m5.high,
            ohlcv_m5.low,
            ohlcv_m5.close,
            ohlcv_m5.volume,
            ohlcv_m5.close_time
        ]

class DummyFactory(AbstractStrategyFactory):
    def create(self, broker, analyser, candles):
        return DummyStrategy(broker, analyser, candles)

# Locate our input file
datadir = os.path.join(os.path.abspath(''), 'data')
datafile = os.path.join(datadir, 'MNQ_week_1min_continuous_UNadjusted_20240425_fixed.csv')
# Declare schema of the input file
schema = CSVCandlesSchema(open_time=0, open=1, high=2,
                          low=3, close=4, volume=5, close_time=6)
# Declare feed
feed = CSVCandlesFactory(datafile, 'MNQUSD', tf.M1, 
                         delimiter=',', schema=schema,
                         timezone='America/New_York')

since = datetime.fromisoformat("2024-03-10 18:00:00-04:00")
until = datetime.fromisoformat("2024-03-10 19:00:00-04:00")

run_backtest(DummyStrategy, DummyFactory(),
             feed, 10_000, since, until, None,
             prefetch_option=PREFETCH_SINCE)

print(m1.head(10), '\n')
print(m5.head(10))

INFO:backintime:Start prefetching...
INFO:backintime:count: 0
INFO:backintime:since: 2024-03-10 18:00:00-04:00
INFO:backintime:until: 2024-03-10 18:00:00-04:00
INFO:backintime:Prefetching is done
INFO:backintime:Start backtesting...
INFO:backintime:Backtesting is done


                          upd_time                 open_time      open      high       low     close volume                       close_time
0 2024-03-10 18:00:59.999000-04:00 2024-03-10 18:00:00-04:00  18300.25   18308.5     18287     18289    494 2024-03-10 18:00:59.999000-04:00
1 2024-03-10 18:01:59.999000-04:00 2024-03-10 18:01:00-04:00  18289.75     18293  18271.75   18290.5    583 2024-03-10 18:01:59.999000-04:00
2 2024-03-10 18:02:59.999000-04:00 2024-03-10 18:02:00-04:00  18288.25     18291   18273.5  18274.25    150 2024-03-10 18:02:59.999000-04:00
3 2024-03-10 18:03:59.999000-04:00 2024-03-10 18:03:00-04:00     18274     18278     18266  18267.25    280 2024-03-10 18:03:59.999000-04:00
4 2024-03-10 18:04:59.999000-04:00 2024-03-10 18:04:00-04:00     18267  18270.25  18248.25  18250.75    389 2024-03-10 18:04:59.999000-04:00
5 2024-03-10 18:05:59.999000-04:00 2024-03-10 18:05:00-04:00  18251.25     18260  18247.75  18251.75    288 2024-03-10 18:05:59.999000-04:00
6 2024-03-10 

## How it works
The class to look at is `CandlesBuffer` and it is located in `src/backintime/candles.py`. It maintaines a mapping where timeframes are keys and current OHLCV for them are the values. It reevaluates OHLCV on invokation of the `update` method. The engine calls the `update` method each time a new candle arrives (see `run_backtest` function in `src/backintime/utils.py`). This behaviour can be demonstrated as follows

In [12]:
from decimal import Decimal
from datetime import datetime, timedelta

from backintime.candles import CandlesBuffer, InputCandle
from backintime.timeframes import Timeframes as tf

start_time = datetime.fromisoformat('2020-01-01 00:00+00:00')
sample_candles = [    # our sample data
    InputCandle(open_time=start_time,
                open=Decimal(1000),
                high=Decimal(1050),
                low=Decimal(950),
                close=Decimal(1050),
                volume=Decimal(100),
                close_time=start_time + timedelta(seconds=59)),
    
    InputCandle(open_time=start_time + timedelta(minutes=1),
                open=Decimal(1100),
                high=Decimal(1150),
                low=Decimal(1050),
                close=Decimal(1150),
                volume=Decimal(100),
                close_time=start_time + timedelta(minutes=1, seconds=59)),

    InputCandle(open_time=start_time + timedelta(minutes=2),
                open=Decimal(1200),
                high=Decimal(1250),
                low=Decimal(1150),
                close=Decimal(1150),
                volume=Decimal(100),
                close_time=start_time + timedelta(minutes=2, seconds=59)),

    InputCandle(open_time=start_time + timedelta(minutes=3),
                open=Decimal(1300),
                high=Decimal(1350),
                low=Decimal(1250),
                close=Decimal(1250),
                volume=Decimal(100),
                close_time=start_time + timedelta(minutes=3, seconds=59)),
    
    InputCandle(open_time=start_time + timedelta(minutes=4),
                open=Decimal(1400),
                high=Decimal(1450),
                low=Decimal(1350),
                close=Decimal(1450),
                volume=Decimal(100),
                close_time=start_time + timedelta(minutes=4, seconds=59)),
]

candles_buffer = CandlesBuffer(start_time, {tf.M1, tf.M5})
for candle in sample_candles:
    candles_buffer.update(candle)

print(f"Current 1MIN: {candles_buffer.get(tf.M1)}")
print(f"Current 5MIN: {candles_buffer.get(tf.M5)}")

Current 1MIN: Candle(open_time=datetime.datetime(2020, 1, 1, 0, 4, tzinfo=datetime.timezone.utc), close_time=datetime.datetime(2020, 1, 1, 0, 4, 59, 999000, tzinfo=datetime.timezone.utc), open=Decimal('1400'), high=Decimal('1450'), low=Decimal('1350'), close=Decimal('1450'), volume=Decimal('100'), is_closed=False)
Current 5MIN: Candle(open_time=datetime.datetime(2020, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), close_time=datetime.datetime(2020, 1, 1, 0, 4, 59, 999000, tzinfo=datetime.timezone.utc), open=Decimal('1000'), high=Decimal('1450'), low=Decimal('950'), close=Decimal('1450'), volume=Decimal('500'), is_closed=False)


---
Note that although it is implemented in the `CandlesBuffer` class, it is `Candles` that get passed to user code (to the trading strategy implementation). The latter just wraps the `CandlesBuffer` classs and only exposes method `get` hiding the `update` method of `CandlesBuffer`.